In [25]:
import yfinance as yf
import pandas as pd

from backtesting import Backtest, Strategy
from backtesting.lib import crossover


In [45]:
data = pd.read_csv("./EURUSD_20Y.csv", parse_dates=True, index_col=0)

data = data.round(decimals = 4)

data = data.dropna()

data


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-03-26,1.2354,1.2449,1.2352,1.2354,1.2354,0.0
2018-03-27,1.2455,1.2478,1.2373,1.2457,1.2457,0.0
2018-03-28,1.2408,1.2422,1.2337,1.2408,1.2408,0.0
2018-03-29,1.2313,1.2338,1.2285,1.2310,1.2310,0.0
2018-03-30,1.2303,1.2334,1.2298,1.2303,1.2303,0.0
...,...,...,...,...,...,...
2023-03-21,1.0719,1.0787,1.0705,1.0719,1.0719,0.0
2023-03-22,1.0774,1.0801,1.0760,1.0774,1.0774,0.0
2023-03-23,1.0870,1.0929,1.0869,1.0870,1.0870,0.0


In [54]:
def EMA(values, n):
    return pd.Series(values).ewm(span=n).mean()


class CrossoverStrategy(Strategy):

    count = 0
    price = 0

    def init(self):
        self.ema55 = self.I(EMA, self.data.Close, 21)
        self.ema200 = self.I(EMA, self.data.Close, 55)

    def next(self):
        price = self.data.Close[-1]

        if self.position.is_long or self.position.is_short:
            # print(dir(self.position))
            print(self.position.pl, self.position.pl_pct, self.position.size)

            if (price*1.04 <= self.price):
                self.position.close()

            # if (price*0.99 >= self.price):
            #     self.position.close()

            print("Price: ", price, self.price, self.trades)

            return

        if crossover(self.ema55, self.ema200):
            if self.count == 0:
                self.count = 1
                return

            self.price = price
            self.buy()


bt = Backtest(data, CrossoverStrategy, cash=100)

output = bt.run()
bt.plot()

print(output)

output['_trades']


0.0 0.0 85
Price:  1.1751 1.1776 (<Trade size=85 time=130- price=1.1751- pl=0>,)
0.016999999999998128 0.0001701982809974023 85
Price:  1.1753 1.1776 (<Trade size=85 time=130- price=1.1751- pl=0>,)
0.10199999999998877 0.0010211896859839698 85
Price:  1.1763 1.1776 (<Trade size=85 time=130- price=1.1751- pl=0>,)
-0.016999999999998128 -0.00017019828099729128 85
Price:  1.1749 1.1776 (<Trade size=85 time=130- price=1.1751- pl=-0>,)
-0.9690000000000065 -0.009701302016849711 85
Price:  1.1637 1.1776 (<Trade size=85 time=130- price=1.1751- pl=-1>,)
-1.2409999999999954 -0.01242447451280737 85
Price:  1.1605 1.1776 (<Trade size=85 time=130- price=1.1751- pl=-1>,)
-1.470500000000008 -0.014722151306271858 85
Price:  1.1578 1.1776 (<Trade size=85 time=130- price=1.1751- pl=-1>,)
-1.6915000000000024 -0.016934728959237533 85
Price:  1.1552 1.1776 (<Trade size=85 time=130- price=1.1751- pl=-2>,)
-2.3375000000000066 -0.023402263637137377 85
Price:  1.1476 1.1776 (<Trade size=85 time=130- price=1.1751-

/home/pavix/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/pavix/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2018-03-26 00:00:00
End                       2023-03-25 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   66.743295
Equity Final [$]                      88.6098
Equity Peak [$]                       100.102
Return [%]                           -11.3902
Buy & Hold Return [%]               -12.87842
Return (Ann.) [%]                   -2.308102
Volatility (Ann.) [%]                5.217732
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -15.71557
Avg. Drawdown [%]                   -15.71557
Max. Drawdown Duration     1641 days 00:00:00
Avg. Drawdown Duration     1641 days 00:00:00
# Trades                                    5
Win Rate [%]                             20.0
Best Trade [%]                       4.302743
Worst Trade [%]                     -5.132123
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,85,130,159,1.1751,1.1403,-2.9580,-0.029615,2018-09-24,2018-11-02,39 days
1,84,209,353,1.1515,1.1082,-3.6372,-0.037603,2019-01-11,2019-08-02,203 days
2,83,417,519,1.1154,1.0688,-3.8678,-0.041779,2019-10-31,2020-03-23,144 days
3,80,570,1067,1.1126,1.0555,-4.5680,-0.051321,2020-06-02,2022-04-28,695 days
4,82,1210,1304,1.0319,1.0763,3.6408,0.043027,2022-11-15,2023-03-25,130 days
